To ensure data integrity and enable more efficient data query, raw fact data is loaded into a table named **raw_clickstream** and then transformed and loaded into **clickstream** table. 

In the data description txt file, description of data attributes is provided. The information is organized into dimension tables and then loaded into PostgreSQL. Each dimension table focuses on a specific aspect of data, such as time, product category, and customer geography.

🌟 By organizing data into fact and dimension tables and creating relationships between these tables, it becomes easier to analyze and update data. This is also a critical part of building a robust and scable <span style="color:blue">data warehouse</span>  to efficiently adapt to changes in data analytics and reporting requirements.

<img title="rawdata" alt="rawdata" src="./rawdata.png">

![rawdata](rawdata.png)

<img title="datamodel" alt="datamodel" src="./datamodel.png">

![datamodel](datamodel.png)

**Steps**:

1. Create a database named ‘OnlineRetail’ in PostgreSQL
2. Create a table in database **raw_clickstream** for the raw clickstream data and load data into PostgreSQL.
3. Create dimension tables and load data into PostgreSQL
4. Transform data in raw_clickstream table and load into processed fact table **clickstream** into PostgreSQL.

**Note**: 
- Avoid using reserved keywords as column names


In [ ]:
-- Create raw fact table

CREATE TABLE raw_clickstream (
    year Int,
	month Int,
	day Int,
	session_order Int,
	country VARCHAR(255),
	session_ID Int,
	page_1 VARCHAR(255),
	page_2 VARCHAR(255),
	color VARCHAR(255),
	location Int,
	model_photography Int,
	price Int,
	price_2 Int,
	page Int
);

In [ ]:
-- Create dimension tables

CREATE TABLE calendar(
    date_id INT NOT NULL PRIMARY KEY,
	day INT NOT NULL CHECK (day BETWEEN 1 AND 31),
    month INT NOT NULL CHECK (month BETWEEN 1 AND 12),
    year INT NOT NULL,
    is_weekend BOOLEAN NOT NULL,
    day_of_week VARCHAR(255) NOT NULL
);
ALTER TABLE calendar ADD COLUMN date DATE;
UPDATE calendar
SET date = TO_DATE(day || '-' || month || '-' || year,'DD-MM-YYYY');

CREATE TABLE country_dim(
	country_id INT PRIMARY KEY,
	country char(255) NOT NULL
);

SELECT * FROM country_dim

CREATE TABLE country_dim(
	country_id INT PRIMARY KEY,
	country char(255) NOT NULL
);

CREATE TABLE category_dim(
	category_id INT PRIMARY KEY,
	category char(255) NOT NULL
);

CREATE TABLE color_dim(
	color_id INT PRIMARY KEY,
	color char(255) NOT NULL
);

CREATE TABLE pagelocation_dim(
	pagelocation_id INT PRIMARY KEY,
	pagelocation char(255) NOT NULL
);

CREATE TABLE model_photography_dim(
	model_photography_id INT PRIMARY KEY,
	model_photography char(255) NOT NULL
);


In [ ]:
-- Create clickstream table and load transformed data

CREATE TABLE clickstream (
	year INT NOT NULL,
 	month INT NOT NULL CHECK (month BETWEEN 1 AND 12),
 	day INT NOT NULL CHECK (day BETWEEN 1 AND 31),
	session_order Int,
	country_id Int,
	session_id Int,
	category_id Int,
 	product_id VARCHAR(255),
    color_id Int,
    pagelocation_id Int,
    model_photography_id Int,
    price Int,
    price_2 Int,
    page_num Int,
    FOREIGN KEY (country_id) REFERENCES country_dim(country_id),
    FOREIGN KEY (category_id) REFERENCES category_dim(category_id),
    FOREIGN KEY (color_id) REFERENCES color_dim(color_id),
    FOREIGN KEY (pagelocation_id) REFERENCES pagelocation_dim(pagelocation_id),
    FOREIGN KEY (model_photography_id) REFERENCES model_photography_dim(model_photography_id)
);

INSERT INTO clickstream (year, month, day,session_order,country_id,session_id,category_id,product_id,color_id,pagelocation_id,model_photography_id,price,price_2,page_num)
SELECT year,month,day,session_order,country,session_id,page_1,page_2,color,location,model_photography,price,price_2,page
FROM raw_clickstream;

ALTER TABLE clickstream ADD COLUMN date_id Int;
UPDATE clickstream
SET date_id = year*10000+month*100+day;